In [1]:

import json
try:
    import json
except:
    %pip install json
    import json

import os

try:
    from sortedcontainers import SortedDict
except:
    %pip install sortedcontainers
    from sortedcontainers import SortedDict

try:
    import plotly.express as px
except:
    %pip install plotly.express
    import plotly.express as px



# Load data file - events_all.json

In [2]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

   # print(number_ex_by_year)

## Histogram - number of exhibitions by year - all

In [3]:

x_values = number_ex_by_year.keys()
y_values = number_ex_by_year.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Year", "y": "Number of exhibitions"},
title="Number of exhibitions by year"
)
fig.show()

# History - exhibitions per year - non MoMa

In [5]:
filepath = "api"
filename = "events_nonmoma.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

    #print(number_ex_by_year)

    x_values = number_ex_by_year.keys()
    y_values = number_ex_by_year.values()

    fig = px.bar(x=x_values, y=y_values, 
    labels={"x": "Year", "y": "Number of exhibitions"},
    title="Number of exhibitions by year - non-MoMa data"
    )
    fig.show()

# A histogram of number of exhibitions per institution !(MoMA+PS1+whitney) 

In [8]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_by_org = {}

    data = json.load(json_file)
    for event in data.get("events"):
        carried_out_by = event.get("carried_out_by")
        for org in carried_out_by: 
            if "Person" in org:
                continue
           

            # get label for id
            org_file = "../data/group/" + org.split("/").pop() + ".json"
            with open(os.path.join(org_file), 'r') as org_file:
                orgdata = json.load(org_file)

                org_label = orgdata.get("_label")

            if org_label in number_by_org:
                number_by_org[org_label] += 1
            else:
                number_by_org[org_label] = 1
        
    number_by_org = SortedDict(number_by_org)

    
    # sort by number
    number_by_org = dict(sorted(number_by_org.items(), key=lambda item: item[1]))

#print(number_by_org)

x_values = number_by_org.keys()
y_values = number_by_org.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Org", "y": "Number of exhibitions"},
title="Number of exhibitions by org"
)
fig.show()

# Remove 'Museum of Modern Art' and 'MoMA PS1'
 

In [9]:
if "The Museum of Modern Art" in number_by_org:
    del number_by_org["The Museum of Modern Art"]
if "MoMA PS1" in number_by_org:
    del number_by_org["MoMA PS1"]

In [11]:
# sort by number
number_by_org = dict(sorted(number_by_org.items(), key=lambda item: item[1]))

x_values = number_by_org.keys()
y_values = number_by_org.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Org", "y": "Number of exhibitions"},
title="Number of exhibitions by organisation"
)
fig.show()


# Number of exhibitions per venue

In [12]:
filepath = "api"
filename = "events_all.json"



with open(os.path.join(filepath, filename), 'r') as json_file:
    events_venue = {}
    number_dict = {}

    data = json.load(json_file)
    for event in data.get("events"):
        venue = event.get("venue")

        
        
        if venue in number_dict:
            number_dict[venue] += 1
            events_venue[venue].append(event)
        else:
            number_dict[venue] = 1
            events_venue[venue] = [event]
        
    number_dict = SortedDict(number_dict)

    
    # sort by number
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    

    for venue in events_venue:
        
        venue_filename = venue.replace(" ", "")
        venue_filename = venue_filename.replace(".", "")
        venue_filename = venue_filename.replace("/", "")
        with open('data/summary/venue/exhibitions/' + venue_filename + '.json', 'w') as ex_file:
            ex_file.write(json.dumps({"meta": {"venue": venue, "exhibition_count": number_dict[venue]},"events": events_venue[venue]}))


    venue_index = []

    for venue in number_dict:
        venue_filename = venue.replace(" ", "")
        venue_filename = venue_filename.replace(".", "")
        venue_filename = venue_filename.replace("/", "")
        venue_index.append({"name": venue, "filename": venue_filename,  "number_exhibitions": number_dict[venue]})

    with open('data/summary/venue/exhibitions/index.json', 'w') as ex_file:
            ex_file.write(json.dumps({"venues": venue_index}))

    

In [13]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Venue", "y": "Number of exhibitions"},
title="Number of exhibitions by venue"
)
fig.show()

In [14]:
if "MoMA" in number_dict:
    del number_dict["MoMA"]
if "MoMA PS1" in number_dict:
    del number_dict["MoMA PS1"]

In [15]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Venue", "y": "Number of exhibitions"},
title="Number of exhibitions by venue"
)
fig.show()

# Number of Artists per Nationality

In [20]:
filepath = "api"
filename = "persons_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_dict = {}

    data = json.load(json_file)
    for person in data.get("persons"):

        if "nationality" in person:
            nat = person.get("nationality")

            if nat in ["","Male", "Female"]:
                continue
            if nat == "American":
                continue
            if nat in number_dict:
                number_dict[nat] += 1
            else:
                number_dict[nat] = 1
        
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    # sort by number
    
   # print(number_dict)

x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Nationality", "y": "Number of people"},
title="Number of people involved in exhibitions by nationality (excluding Americans and empty values)"
)
fig.show()

# Gender

In [22]:
filepath = "api"
filename = "persons_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_dict = {}

    data = json.load(json_file)
    for person in data.get("persons"):

        if "nationality" in person:
            gender = person.get("nationality")

            if gender not in ["Male", "Female"]:
                continue
           
            if gender in number_dict:
                number_dict[gender] += 1
            else:
                number_dict[gender] = 1
        
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    # sort by number
    
   # print(number_dict)

x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Gender", "y": "Number of people"},
title="Number of people involved in exhibitions by gender"
)
fig.show()

# Number of exhibitions per year for each organisation

In [24]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")
        start = event.get("start")
        year = start.split("-")[0]

        if org in number_ex_by_year:

            if year in number_ex_by_year[org]:
                number_ex_by_year[org][year] += 1
            else:
                number_ex_by_year[org][year] = 1
        else:
            number_ex_by_year[org] = {}
            number_ex_by_year[org][year] = 1
        
    #number_ex_by_year = SortedDict(number_ex_by_year)

    

    for org in number_ex_by_year:

        numbers = number_ex_by_year[org]
        numbers = SortedDict(numbers)
        #print(numbers)
        x_values = numbers.keys()
        y_values = numbers.values()

        fig = px.bar(x=x_values, y=y_values, 
        labels={"x": "Year", "y": "Number of exhibitions"},
        title="Number of exhibitions by year - " + org
        )
        fig.show()
        